In [1]:
import os
import sys
# Add the scripts folder to the Python path
sys.path.append(os.path.abspath("../scripts"))

In [2]:
import yfinance as yf 
import pandas as pd 
import numpy as np
from datetime import datetime  
from keras.models import Sequential
from keras.layers import Dense, LSTM

In [3]:
end=datetime.now()
start=datetime(end.year-15,end.month,end.day)
stock=('BTC-USD')
stock_data=yf.download(stock,start=start,end=end)

[*********************100%***********************]  1 of 1 completed


In [4]:
#closing price history
closing_price=stock_data[['Close']]

In [5]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(closing_price[['Close']].dropna())

In [7]:
len(scaled_data)

3773

In [8]:
#prepare data for LSTM  
x_data=[]
y_data =[]
base_days=100
for i in range(base_days,len(scaled_data)):
    x_data.append(scaled_data[i-base_days: i])
    y_data. append(scaled_data[i])
x_data= np.array(x_data)
y_data= np.array(y_data)

In [9]:
#split data into training and testing
train_size= int(len(x_data)*0.9)
x_train,y_train=x_data[:train_size],y_data[:train_size]
x_test,y_test=x_data[train_size:],y_data[train_size:] 